# 🎥 YouTube Livestream Summarizer

AI-Powered Real-Time Livestream Analysis using FFmpeg and Google's Gemini AI.

**GitHub Repository**: [youtube-livestream-summarizer](https://github.com/leoparker157/youtube-livestream-summarizer)

---

## Features
- 🚀 Real-time livestream recording with optimized compression
- 🤖 AI-powered summarization using Gemini 2.0 Flash
- ⚡ Speed-optimized for continuous processing
- 📊 Customizable video duration and segments
- 📝 Live progress monitoring and summary display

---

## 🎮 Step 0: Enable GPU (T4 Recommended)

**IMPORTANT**: This app requires a GPU for fast video encoding.

1. Click **Runtime** → **Change runtime type**
2. Set **Hardware accelerator** to **T4 GPU**
3. Click **Save**

The next cell will verify GPU is enabled.

## ⚡ FFmpeg Buffer Optimization

This notebook includes automatic buffer handling to prevent FFmpeg crashes after 2-3 minutes.

**What we've optimized:**
- ✅ Proper stdout/stderr draining to prevent buffer overflow
- ✅ Reduced FFmpeg logging to prevent output buildup
- ✅ HLS reconnection flags for network stability
- ✅ Background thread output handling
- ✅ Memory-efficient segment processing

These fixes ensure FFmpeg can run for extended periods without freezing or crashing.

In [ ]:
# Check if GPU is available
import subprocess
import sys

print("🔍 Checking GPU availability...")
print("="*50)

# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running on Google Colab")
except:
    IN_COLAB = False
    print("⚠️ Not running on Google Colab")

# Check NVIDIA GPU
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("\n🎮 GPU DETECTED!")
        print("="*50)
        # Parse nvidia-smi output to show GPU info
        lines = result.stdout.split('\n')
        for line in lines:
            if 'Tesla' in line or 'T4' in line or 'GPU' in line:
                print(line.strip())
        print("="*50)
        print("✅ NVENC hardware encoding will be used (FAST)")
    else:
        print("\n❌ NO GPU DETECTED!")
        print("="*50)
        print("⚠️ CPU encoding will be used (SLOW)")
        print("\n💡 To enable GPU:")
        print("   1. Click Runtime → Change runtime type")
        print("   2. Set Hardware accelerator to 'T4 GPU'")
        print("   3. Click Save and restart")
        print("="*50)
except FileNotFoundError:
    print("\n❌ NO GPU DETECTED!")
    print("="*50)
    print("⚠️ nvidia-smi not found - CPU encoding will be used (SLOW)")
    if IN_COLAB:
        print("\n💡 To enable GPU:")
        print("   1. Click Runtime → Change runtime type")
        print("   2. Set Hardware accelerator to 'T4 GPU'")
        print("   3. Click Save and restart")
    print("="*50)

print("\n✅ Ready to proceed!")

## 📦 Step 1: Install Dependencies

This cell installs all required packages including FFmpeg, yt-dlp, and Python libraries.

In [ ]:
%%capture
# Install system dependencies
!apt-get update -qq
!apt-get install -y ffmpeg > /dev/null 2>&1

# Install Python packages
!pip install -q gradio google-generativeai python-dotenv schedule yt-dlp

print("✅ All dependencies installed successfully!")

## 🔑 Step 2: Set Your Gemini API Key

Get your free API key from [Google AI Studio](https://aistudio.google.com/app/apikey)

**Note**: You can also enter it directly in the Gradio interface.

In [ ]:
import os

# Option 1: Set API key here (recommended for Colab)
GEMINI_API_KEY = ""  # Paste your API key here

# Option 2: Use Colab secrets (more secure)
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    if GEMINI_API_KEY:
        print("✅ API key set manually")
    else:
        print("⚠️ No API key set. You can enter it in the Gradio interface.")

os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

## 📥 Step 3: Clone Repository and Load Application

This downloads the latest version of the summarizer from GitHub.

In [ ]:
# Clone the repository
!git clone https://github.com/leoparker157/youtube-livestream-summarizer.git
%cd youtube-livestream-summarizer

print("✅ Repository cloned successfully!")

## 🚀 Step 4: Launch Gradio Interface

This will start the web interface. Click the public URL to access it.

**The interface includes:**
- YouTube URL input
- API key configuration
- Advanced settings (video duration, segments, overlap)
- Custom prompt editor
- Real-time progress log
- Live summary display

In [ ]:
# Import and run the Gradio app
import sys
sys.path.append('.')

from gradio_app import create_interface

# Create and launch the interface
demo = create_interface()
demo.launch(share=True, debug=True)

---

## 📖 How to Use

1. **Enable GPU**: Make sure T4 GPU is enabled (see Step 0)
2. **Get API Key**: Visit [Google AI Studio](https://aistudio.google.com/app/apikey) and get a free API key
3. **Enter YouTube URL**: Paste a YouTube livestream URL (must be live or streaming)
4. **Configure Settings** (optional):
   - **Video Clip Duration**: How long each video clip should be (60-300 seconds)
   - **Segment Duration**: Size of recording segments (5-60 seconds)
   - **Overlap Segments**: Number of overlapping segments between cycles
   - **Custom Prompt**: Customize how Gemini summarizes the content
5. **Start**: Click "Start Recording & Summarizing"
6. **Monitor**: Watch real-time progress in the left panel
7. **View Summaries**: See AI-generated summaries appear in the right panel
8. **Stop**: Click "Stop" when finished

---

## ⚙️ Configuration Examples

### Fast Processing (Quick Summaries)
- Video Clip Duration: 60 seconds
- Segment Duration: 30 seconds
- Overlap: 0
- **Recommended for**: Short streams, limited resources

### Standard Processing (Balanced) ⭐ Recommended
- Video Clip Duration: 120 seconds
- Segment Duration: 30 seconds
- Overlap: 0
- **Recommended for**: Most use cases

### Deep Analysis (Detailed Summaries)
- Video Clip Duration: 180-240 seconds
- Segment Duration: 30 seconds
- Overlap: 0
- **Recommended for**: Long-form content, detailed analysis

**Note**: Longer durations give more context but take longer to process. Start with Standard settings.

---

## 🔧 FFmpeg Stability Improvements

This notebook includes several optimizations to prevent FFmpeg crashes:

### Buffer Overflow Prevention
- **Output draining**: FFmpeg stderr is captured and drained to prevent buffer buildup
- **Reduced logging**: Uses `-loglevel error` to minimize output
- **Background threads**: Output is handled asynchronously

### Network Stability
- **Reconnection flags**: Automatic HLS stream reconnection
  - `-reconnect 1`: Enable reconnection
  - `-reconnect_streamed 1`: Reconnect to HLS streams
  - `-reconnect_delay_max 10`: Max 10s delay between reconnects
  - `-timeout 30000000`: 30 second timeout

### Resource Management
- **Automatic cleanup**: Old segments are deleted automatically
- **Memory-efficient**: Only keeps necessary segments in memory
- **GPU encoding**: Uses NVENC for fast, low-CPU encoding

### Monitoring
- **Process health checks**: Detects if FFmpeg exits unexpectedly
- **Segment size tracking**: Monitors recording progress
- **Error diagnostics**: Shows detailed error messages when issues occur

---

## 📝 Custom Prompt Examples

### News/Politics Stream
```
Summarize the key topics discussed in this news stream:
1. Main headlines and breaking news
2. Guest speakers and their opinions
3. Any controversies or debates
4. Use clear, concise language
```

### Gaming Stream
```
Summarize this gaming stream:
1. What game is being played
2. Key moments, wins, or losses
3. Streamer's reactions and commentary
4. Any interesting interactions with chat
5. Keep it casual and fun
```

### Educational/Tutorial Stream
```
Create a structured summary of this educational content:
1. Main topic being taught
2. Key concepts explained
3. Important tips or warnings
4. Practical examples shown
5. Use bullet points for clarity
```

---

## ⚠️ Important Notes

- **GPU Required**: T4 GPU is REQUIRED for proper operation. CPU encoding is too slow for livestreams.
- **Session Limits**: Google Colab free tier has ~12 hour session limits. Pro/Pro+ have longer limits.
- **HLS URL Expiration**: YouTube HLS URLs expire after a few minutes. The app will detect this and show a clear error message.
- **API Limits**: Free Gemini API has rate limits. Consider upgrading for heavy usage.
- **Network**: Stable internet connection required for streaming.
- **Storage**: Segments are cleaned automatically to save disk space.

### Expected Behavior
- **First cycle**: Takes ~2-5 minutes (needs 4 segments @ 30s each)
- **Subsequent cycles**: Every 2 minutes (4 new segments)
- **FFmpeg runs continuously**: Recording never stops between cycles
- **Summaries in background**: Processing happens while recording continues

### Common Warnings (Normal)
- ⚠️ "segment_XXX.mp4 not started yet" - Normal wait between cycles
- 🔄 "Finalizing: segment_XXX.mp4" - Last segment completing
- 📊 "Cycle #X: Starting..." - New cycle beginning

---

## 🔧 Troubleshooting

### FFmpeg Exits After 2-3 Minutes (Exit Code 0)
- **Cause**: YouTube HLS URL expired (URLs expire after a few minutes)
- **Solution**: This is expected behavior. The app will show detailed diagnostics.
- **Current Status**: Automatic HLS URL refresh is not yet implemented.
- **Workaround**: 
  1. Use shorter video durations (60-90s instead of 120s)
  2. Restart manually every ~2 minutes
  3. Wait for automatic refresh feature (coming soon)

### "No GPU detected" or very slow recording
- **Solution**: Enable T4 GPU runtime
- Go to Runtime → Change runtime type → Set to T4 GPU
- Restart the runtime and run all cells again
- **Note**: CPU encoding is too slow for real-time livestreams

### "Failed to extract HLS URL"
- Make sure the YouTube URL is correct
- Ensure the stream is live or available
- Try updating yt-dlp: `!pip install -U yt-dlp`
- Check if the stream requires authentication

### "Gemini API Error"
- Check that your API key is valid
- Verify you haven't exceeded rate limits
- Try a shorter video duration
- Wait a few minutes and try again

### Segments Stuck at Same Size
- Check FFmpeg output (shown in progress log when FFmpeg exits)
- Verify stream is still live
- Check network connection
- Restart and try again with shorter segments

### "Out of Memory"
- Reduce video clip duration (try 60-90 seconds)
- Decrease segment duration (try 15-20 seconds)
- Use Colab Pro for more RAM (25GB vs 12GB)
- Close other browser tabs

### Recording Seems Stuck
- Check "Recording: segment_XXX.mp4" line - file size should grow
- If size is increasing, recording is working (be patient)
- First cycle takes longer (needs to accumulate initial segments)
- Use shorter segment duration for faster cycles

---

## 🧹 Optional: Cleanup

Run this cell to clean up files after you're done.

In [ ]:
# Clean up temporary files
!rm -rf segments/
!rm -f compressed.mp4
!rm -f *.txt

print("✅ Cleanup complete!")